In [1]:
import os

In [2]:
%pwd

'/Users/palakprashant/E2E_ML_Project_MLflow/research'

In [3]:
os.chdir("../") #going back 1 folder

In [4]:
%pwd

'/Users/palakprashant/E2E_ML_Project_MLflow'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfiguration:
    root_directory: Path
    source_URL: str
    local_data: Path
    unzip_directory: Path

In [6]:
from ml_proj.constants import *
from ml_proj.utils.common import read_yaml, create_directories

In [21]:
class ConfigManager:
    def __init__(
            self,
            config_filepath = config_file,
            params_filepath = params_file,
            schema_filepath = schema_file):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_source])

    def get_data_ingestion_configuration(self) -> DataIngestionConfiguration:
        configuration = self.config.data_ingestion
        create_directories([configuration.root_directory])

        data_ingestion_configuration = DataIngestionConfiguration(
            root_directory= configuration.root_directory,
            source_URL= configuration.source_URL,
            local_data= configuration.local_data,
            unzip_directory=configuration.unzip_directory
        )
        return data_ingestion_configuration

In [22]:
import os
import urllib.request as request #to retrieve and download data from the URL
import zipfile #to unzip data
from ml_proj import logger
from ml_proj.utils.common import size


In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfiguration):
        self.config = config
    
    def download_data(self):
        if not os.path.exists(self.config.local_data):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data
            )
            logger.info(f"{filename} download with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {size(Path(self.config.local_data))}")
    
    def extract_zip(self):
        #This function extracts the zip file into the data directory with None return type
        unzip_path = self.config.unzip_directory
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data, 'r') as zip_reference:
            zip_reference.extractall(unzip_path)

In [ ]:
#Update pipeline
try:
    configuration = ConfigManager()
    data_ingestion_configuration = configuration.get_data_ingestion_configuration()
    data_ingestion = DataIngestion(config = data_ingestion_configuration)
    data_ingestion.download_data()
    data_ingestion.extract_zip()
except Exception as e:
    raise e

[2024-11-06 17:05:14,442: INFO: common: yaml file: src/ml_proj/config/config.yaml loaded successfully.]
[2024-11-06 17:05:14,443: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-11-06 17:05:14,444: INFO: common: yaml file: schema.yaml loaded successfully.]
[2024-11-06 17:05:14,445: INFO: common: Created directory here: artifacts]
[2024-11-06 17:05:14,445: INFO: common: Created directory here: artifacts/data_ingestion]
[2024-11-06 17:05:14,923: INFO: 1830773562: artifacts/data_ingestion/data.zip download with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7FC8:1E0811:47F213:4DD9A0:672BE81A
Accept-Ranges: b